---
title: "Data Analysis"
subtitle: "Comprehensive Data Cleaning & Exploratory Analysis of Job Market Trends"
author:
  - name: Furong Wang
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
  - name: Marco Perez Garcia
    affiliations:
      - ref: bu
bibliography: references.bib
csl: csl/econometrica.csl
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
---